In [70]:
import os
import string
import numpy as np
import pandas as pd

from collections import Counter
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import load, dump

from sys import path
path.append("../utils/")

from utils import *

In [2]:
DATASETS = ["20ng", "acm", "webkb", "reut"]

DTESTS = ["webkb"]

ALGORITHMS = ["centroide", "gbm", "knn", "lr", "rf", "svm", "bert"]

BERT_ALGS = ["gbm", "knn", "lr", "rf", "svm"]

In [3]:
CLF_ID = { clf:idx for clf, idx in zip(ALGORITHMS, range(len(ALGORITHMS)))}
CLF_ID

{'centroide': 0, 'gbm': 1, 'knn': 2, 'lr': 3, 'rf': 4, 'svm': 5, 'bert': 6}

In [9]:
pd_datasets = {}
for dset in DATASETS:
    pd_datasets[dset] = pd.read_csv(f"../../../stacking/output/datasets/{dset}.csv")

In [19]:
#hits_count = np.sum(df.values == pd_datasets["webkb"].classes.values[:, None], axis=1)

In [82]:
with open("/home/welton/data/stop_words_english.txt") as fd:
    stop_words = {}
    for w in fd.read().split('\n'):
        stop_words[w] = True

In [80]:
def is_a_number(text):
    try:
        float(text)
        return True
    except:
        return False

def clean_text(text, stop_words):

    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ' '.join([ word for word in text.split(' ') if word not in stop_words and not is_a_number(word)])
    return text

In [25]:
clean_text("Don't hate the player, hate the game. 000. 000f", stop_words)

'dont hate player hate game 000f'

In [86]:
def get_feature_importance(dset, df, X, fold):

    models_dir = f"data/feature_importance/rf_models/{dset}/"
    os.makedirs(models_dir, exist_ok=True)
    model_path = f"{models_dir}/{fold}.joblib"
    if os.path.exists(model_path):
        return load(model_path)
    else:
        train = df[df.folds_id != fold]
        test = df[df.folds_id == fold]
        X_train, y_train = X[train.index], df.classes.values[train.index]
        rf = RandomForestClassifier(random_state=42, n_jobs=4)
        rf.fit(X_train, y_train)
        dump(rf, model_path)
        return rf

def word_importance(doc, tf, rf):

    tokens = doc.split(' ')
    return [ rf.feature_importances_[tf.vocabulary_[word]] if word in tf.vocabulary_ else 0 for word in tokens ]

def fast_word_importance(doc, tf, feats_name, rf):

    x = tf.transform([doc]).toarray()[0]
    x[x > 0] = 1
    fi = x * rf.feature_importances_
    return [ feats_name[idx] for idx in np.argsort(-fi)[:15] ]


def docs_word_importance(df, tf, rf):

    docs_wi = {}
    feats_name = tf.get_feature_names_out()
    for doc in tqdm(df.itertuples()):
        #docs_wi[doc.index] = word_importance(doc.clean_docs, tf, rf)
        docs_wi[doc.Index] = fast_word_importance(doc.clean_docs, tf, feats_name, rf)
    return docs_wi

In [87]:
def get_docs_word_importance(pd_datasets, DATASETS, stop_words):
    
    docs_wi = {}
    for dset in DATASETS:
        
        print(f"{dset.upper()}")
        docs_wi[dset] = {}
        df = pd_datasets[dset]
        df["clean_docs"] = df.docs.apply(lambda t: clean_text(t, stop_words))

        tf = TfidfVectorizer(min_df=2)
        X = tf.fit_transform(df.clean_docs).toarray()

        for fold in np.arange(10):
            print(f"FOLD: {fold}", end="\r")
            rf = get_feature_importance(dset, df, X, fold)
            missed_docs = df[(df.folds_id == fold) & (df.hits_count == 0)].copy(deep=True)
            docs_wi[dset][fold] = docs_word_importance(missed_docs, tf, rf)    
    return docs_wi

In [88]:
docs_wi = get_docs_word_importance(pd_datasets, DATASETS, stop_words)

20NG


32it [00:07,  4.25it/s]


46it [00:10,  4.34it/s]


35it [00:08,  4.04it/s]


40it [00:08,  4.46it/s]


...

90it [00:14,  6.21it/s]


70it [00:11,  6.25it/s]


75it [00:12,  6.17it/s]


89it [00:14,  6.05it/s]


79it [00:12,  6.39it/s]


In [108]:
import pickle
with open("data/words_importance.pickle", "wb") as fd:
    pickle.dump(docs_wi, fd)

In [4]:
with open("data/words_importance.pickle", "rb") as fd:
    docs_wi = pickle.load(fd)

In [26]:
labels_desc = {}
for dset in DATASETS:
    labels_desc[dset] = {}
    with open(f"data/class_desc/{dset}.txt", 'r') as fd:
        for idx, label in enumerate(fd.read().split('\n')):
            labels_desc[dset][idx] = label


In [106]:
def topword_class(pd_datasets, dset, docs_wi, prefix="topword_class"):

    doc_word_list = []
    df = pd_datasets[dset]
    for fold in docs_wi[dset]:
        docs = docs_wi[dset][fold]
        for doc in docs:
            doc_info = df.iloc[doc]
            y, so_pred = doc_info["classes"], doc_info["conc_pred"]
            doc_word_list.append([
                fold,
                doc,
                ', '.join(docs[doc]),
                f"{y} -- {labels_desc[dset][y]}",
                f"{so_pred} -- {labels_desc[dset][so_pred]}"
            ])
    
    pd.DataFrame(doc_word_list, columns=["Fold", "DocID", "TOP15", "Label", "Stacking"]).to_excel(f"data/{prefix}/{dset}.xlsx")

In [103]:
for dset in DATASETS:
    topword_class(pd_datasets, dset, docs_wi)

In [34]:
bert_preds = load_preds(DATASETS, ALGORITHMS)

LOADING PREDS FROM SCRATCH


280it [00:00, 382.88it/s]


In [110]:
def get_bert_datasets(DATASETS, bert_preds, pd_datasets):
    bert_datasets = {}
    for dset in DATASETS:
        df = pd.DataFrame(bert_preds[dset])
        hits_count = np.sum(df.values == pd_datasets[dset].classes.values[:, None], axis=1)
        folds_id = pd_datasets[dset].folds_id.values
        conc_pred = np.array([ Counter(row).most_common()[0][0] for row in df.values ])
        data = np.vstack([folds_id, conc_pred, hits_count, pd_datasets[dset].docs.values, pd_datasets[dset].classes.values]).T
        bert_datasets[dset] = pd.DataFrame(data, columns=["folds_id", "conc_pred", "hits_count", "docs", "classes"])
    return bert_datasets

In [111]:
bert_datasets = get_bert_datasets(DATASETS, bert_preds, pd_datasets)

In [104]:
bert_docs_wi = get_docs_word_importance(bert_datasets, DATASETS, stop_words)

20NG


170it [00:27,  6.13it/s]


219it [00:38,  5.76it/s]


177it [00:29,  6.02it/s]


184it [00:26,  6.91it/s]


...

220it [00:33,  6.59it/s]


1279it [03:08,  6.78it/s]


173it [00:25,  6.71it/s]


184it [00:26,  6.85it/s]


185it [00:26,  6.91it/s]


In [112]:
for dset in DATASETS:
    topword_class(bert_datasets, dset, bert_docs_wi, prefix="bert_topword_class")